# Depêndencias iniciais
Iremos agora importar as principais bibliotecas:

In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.utils import np_utils
from sklearn.metrics import f1_score, recall_score, precision_score
import json
import os
import glob

# O dataset
Nessa primeira parte, iremos carregar o _dataset_ na estrutura de dados que utilizaremos, que será o [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [27]:
# Lendo arquivo CSV
filename = '/content/drive/My Drive/Ensino/UFRN 2016.1/2020.6/Aprendizado Profundo/projeto_final/fer2013.csv'
df = pd.read_csv(filename)

In [28]:
# Conhecendo a quantidade de linhas e colunas do dataset, respectivamente
print("dataset:= ", df.shape)

dataset:=  (35887, 3)


In [29]:
df.columns

Index(['emotion', 'pixels', 'Usage'], dtype='object')

## Explorando o dataset
Aqui iremos utilizar alguns métodos do pandas para conhecermos melhor o nosso _DataFrame_. Os métodos serão:
- **[head](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html)**: mostra as primeiras _n_ linhas do nosso _dataset_. Por padrão serão as 5 primeiras linhas.
- **[info](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html)**: imprime informações sobre o nosso _DataFrame_, incluindo o tipo de índice, os tipos de coluna, valores não nulos e uso de memória.

- **[describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)**: mostra estatísticas descritivas sobre as colunas do nosso _DataFrame_, como: a tendência central, a dispersão e a forma da distribuição de um conjunto de dados.

In [30]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [32]:
df.describe()

,emotion
count,35887.000000
mean,3.323265
std,1.873819
min,0.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,6.000000


# Modelos


In [33]:
X_train, train_y, X_test, test_y = [], [], [], []

for index, row in df.iterrows():
    val = row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val, 'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val, 'float32'))
            test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

num_labels = 7
width, height = 48, 48

X_train = np.array(X_train, 'float32')
train_y = np.array(train_y, 'float32')
X_test = np.array(X_test, 'float32')
test_y = np.array(test_y, 'float32')

train_y = np_utils.to_categorical(train_y, num_classes=num_labels)
test_y = np_utils.to_categorical(test_y, num_classes=num_labels)

X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [34]:
models = {}

## Convolutional Neural Networks for Facial Expression Recognition

In [35]:
class ShallowCNN(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Convolutional Layer
        self.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), input_shape=(input_shape)))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.25))
        self.add(tf.keras.layers.Activation('relu'))

        # 2nd Convolutional Layer
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1)))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.25))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        self.add(tf.keras.layers.Activation('relu'))

        # Flattening
        self.add(tf.keras.layers.Flatten())

        # Fully connected layers
        self.add(tf.keras.layers.Dense(512))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Activation('relu'))
        self.add(tf.keras.layers.Dropout(0.25))

        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['ShallowCNN'] = ShallowCNN(X_train.shape[1:], num_labels)

In [36]:
class DeepCNN(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), input_shape=(input_shape)))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.25))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        self.add(tf.keras.layers.Activation('relu'))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(5, 5), strides=(1, 1)))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.25))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        self.add(tf.keras.layers.Activation('relu'))

        # 3rd Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1)))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.25))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        self.add(tf.keras.layers.Activation('relu'))

        # 4th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1)))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.25))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
        self.add(tf.keras.layers.Activation('relu'))

        # Flattening
        self.add(tf.keras.layers.Flatten())

        # Fully connected layers
        self.add(tf.keras.layers.Dense(256))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.25))
        self.add(tf.keras.layers.Activation('relu'))

        # Fully connected layer 2nd layer
        self.add(tf.keras.layers.Dense(512))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.25))
        self.add(tf.keras.layers.Activation('relu'))

        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['DeepCNN'] = DeepCNN(X_train.shape[1:], num_labels)

## (Custom) Convolutional Neural Networks for Facial Expression Recognition

In [37]:
class DeepCNNCustom(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), input_shape=(input_shape)))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.2))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        self.add(tf.keras.layers.Activation('relu'))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(5, 5), strides=(1, 1)))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.2))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        self.add(tf.keras.layers.Activation('relu'))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1)))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.2))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        self.add(tf.keras.layers.Activation('relu'))

        # 3rd Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1)))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.3))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
        self.add(tf.keras.layers.Activation('relu'))

        # 4th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same'))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.3))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
        self.add(tf.keras.layers.Activation('relu'))

        # Flattening
        self.add(tf.keras.layers.Flatten())

        # Fully connected layers
        self.add(tf.keras.layers.Dense(256))
        #self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.2))
        self.add(tf.keras.layers.Activation('relu'))

        # Fully connected layer 2nd layer
        self.add(tf.keras.layers.Dense(512))
        #self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.Dropout(0.2))
        self.add(tf.keras.layers.Activation('relu'))

        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['DeepCNNCustom'] = DeepCNNCustom(X_train.shape[1:], num_labels)

## Very Deep Convolutional Networks for Large-Scale Image Recognition

In [38]:
class VGGA11(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(input_shape)))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 3rd Conv block
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 4th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 5th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # Fully connected layers
        self.add(tf.keras.layers.Flatten())
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['VGGA11'] = VGGA11(X_train.shape[1:], num_labels)

In [39]:
class VGGALRN11(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(input_shape)))
        self.add(tf.keras.layers.LayerNormalization())
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 3rd Conv block
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 4th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 5th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # Fully connected layers
        self.add(tf.keras.layers.Flatten())
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['VGGALRN11'] = VGGALRN11(X_train.shape[1:], num_labels)

In [40]:
class VGGB13(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(input_shape)))
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 3rd Conv block
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 4th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 5th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # Fully connected layers
        self.add(tf.keras.layers.Flatten())
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['VGGB13'] = VGGB13(X_train.shape[1:], num_labels)

In [41]:
class VGGC16(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(input_shape), padding='same'))
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 3rd Conv block
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(1, 1), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 4th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(1, 1), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 5th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(1, 1), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # Fully connected layers
        self.add(tf.keras.layers.Flatten())
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['VGGC16'] = VGGC16(X_train.shape[1:], num_labels)

In [42]:
class VGGD16(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(input_shape), padding='same'))
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 3rd Conv block
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 4th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 5th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # Fully connected layers
        self.add(tf.keras.layers.Flatten())
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['VGGD16'] = VGGD16(X_train.shape[1:], num_labels)

In [43]:
class VGGE19(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(input_shape)))
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 3rd Conv block
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 4th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 5th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # Fully connected layers
        self.add(tf.keras.layers.Flatten())
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['VGGE19'] = VGGE19(X_train.shape[1:], num_labels)

## (Custom) Very Deep Convolutional Networks for Large-Scale Image Recognition

In [63]:
class VGGCustom(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(input_shape)))
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        self.add(tf.keras.layers.Dropout(0.5))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        self.add(tf.keras.layers.Dropout(0.5))

        # 3rd Conv block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # Fully connected layers
        self.add(tf.keras.layers.Flatten())
        self.add(tf.keras.layers.Dense(1024, activation='relu'))
        self.add(tf.keras.layers.Dropout(0.2))
        self.add(tf.keras.layers.Dense(1024, activation='relu'))
        self.add(tf.keras.layers.Dropout(0.2))
        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['VGGCustom'] = VGGCustom(X_train.shape[1:], num_labels)

In [44]:
class VGGE19Custom(tf.keras.models.Sequential):
    def __init__(self, input_shape, num_labels):
        super().__init__()

        # 1st Conv Block
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(input_shape)))
        self.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 2nd Conv Block
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(5, 5), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 3rd Conv block
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 4th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(5, 5), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # 5th Conv block
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(5, 5), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
        self.add(tf.keras.layers.BatchNormalization())
        self.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

        # Fully connected layers
        self.add(tf.keras.layers.Flatten())
        self.add(tf.keras.layers.Dense(4096, activation='relu'))
        self.add(tf.keras.layers.Dropout(0.2))
        self.add(tf.keras.layers.Dense(1024, activation='relu'))
        self.add(tf.keras.layers.Dense(num_labels, activation='softmax'))


models['VGGE19Custom'] = VGGE19Custom(X_train.shape[1:], num_labels)

## Treinamento

In [64]:
models

{'DeepCNN': <__main__.DeepCNN at 0x7f3f4da86be0>,
 'DeepCNNCustom': <__main__.DeepCNNCustom at 0x7f3fc010b320>,
 'ShallowCNN': <__main__.ShallowCNN at 0x7f3f4dd167f0>,
 'VGGA11': <__main__.VGGA11 at 0x7f3f6c1c0e10>,
 'VGGALRN11': <__main__.VGGALRN11 at 0x7f3f4da27358>,
 'VGGB13': <__main__.VGGB13 at 0x7f3f4d995a20>,
 'VGGC16': <__main__.VGGC16 at 0x7f3f4d9028d0>,
 'VGGD16': <__main__.VGGD16 at 0x7f3f4d8f5550>,
 'VGGD16Custom': <__main__.VGGD16Custom at 0x7f3f4d98a9e8>,
 'VGGE19': <__main__.VGGE19 at 0x7f3f4d878160>,
 'VGGE19Custom': <__main__.VGGE19Custom at 0x7f3f4d7ba5f8>}

In [65]:
for model in models.values():
  print(model.summary())
  print("\n-------------------------------------\n")

Model: "shallow_cnn_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 46, 46, 32)        320       
_________________________________________________________________
batch_normalization_31 (Batc (None, 46, 46, 32)        128       
_________________________________________________________________
dropout_33 (Dropout)         (None, 46, 46, 32)        0         
_________________________________________________________________
activation_32 (Activation)   (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 44, 44, 64)        18496     
_________________________________________________________________
batch_normalization_32 (Batc (None, 44, 44, 64)        256       
_________________________________________________________________
dropout_34 (Dropout)         (None, 44, 44, 64)      

In [47]:
dirname = '/content/drive/My Drive/Ensino/UFRN 2016.1/2020.6/Aprendizado Profundo/projeto_final/models'

def save_model(model, name, BATCH_SIZE, EPOCHS, hist):
    fer_json = model.to_json()

    with open('{}/{}_{}_{}.json'.format(dirname, name, BATCH_SIZE, EPOCHS), 'w') as json_file:
        json_file.write(fer_json)
    model.save_weights('{}/{}_{}_{}.h5'.format(dirname, name, BATCH_SIZE, EPOCHS))

    with open('{}/hist_{}_{}_{}.json'.format(dirname, name, BATCH_SIZE, EPOCHS), 'w') as json_file:
        json.dump(hist, json_file)
    

def check_model_exists(name, BATCH_SIZE, EPOCHS):
    return os.path.exists('{}/{}_{}_{}.json'.format(dirname, name, BATCH_SIZE, EPOCHS))

In [48]:
from keras import backend as keras_backend

def recall_m(y_true, y_pred):
    true_positives = keras_backend.sum(keras_backend.round(keras_backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = keras_backend.sum(keras_backend.round(keras_backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + keras_backend.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = keras_backend.sum(keras_backend.round(keras_backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = keras_backend.sum(keras_backend.round(keras_backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + keras_backend.epsilon())
    return precision

def f1_metrics(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+keras_backend.epsilon()))

In [66]:
# Training the model
for name, model in models.items():
  print("\n-------------------------------------\n")
  model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[
                tf.keras.metrics.Accuracy(name='accuracy'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                f1_metrics,
                #'accuracy',
                #precision_m,
                #recall_m,
              ])
  
  for BATCH_SIZE in [128, 256]:
    for EPOCHS in [30, 50, 100]:
      print("{} | batch_size: {} | epochs: {}".format(name, BATCH_SIZE, EPOCHS))

      if check_model_exists(name, BATCH_SIZE, EPOCHS):
        continue

      hist = model.fit(X_train, train_y,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                verbose=1,
                validation_data=(X_test, test_y),
                shuffle=True
              )
      
      model_hist = {
          'id': '{}_{}_{}'.format(name, BATCH_SIZE, EPOCHS),
          'name': name,
          'metrics': {
              'loss': hist.history['loss'][-1],
              'val_loss': hist.history['val_loss'][-1],
              'accuracy': hist.history['accuracy'][-1],
              'val_accuracy': hist.history['val_accuracy'][-1],
              'f1_metrics': hist.history['f1_metrics'][-1],
              'val_f1_metrics': hist.history['val_f1_metrics'][-1],
              'precision': hist.history['precision'][-1],
              'val_precision': hist.history['val_precision'][-1],
              'recall': hist.history['recall'][-1],
              'val_recall': hist.history['val_recall'][-1],
          },
          'params': {
              'batch_size': BATCH_SIZE,
              'epochs': EPOCHS,
              'steps': hist.params['steps'],
          }
      }
  
      save_model(model, name, BATCH_SIZE, EPOCHS, model_hist)
      #break
    #break
  #break




-------------------------------------

ShallowCNN | batch_size: 128 | epochs: 30
ShallowCNN | batch_size: 128 | epochs: 50
ShallowCNN | batch_size: 128 | epochs: 100
ShallowCNN | batch_size: 256 | epochs: 30
ShallowCNN | batch_size: 256 | epochs: 50
ShallowCNN | batch_size: 256 | epochs: 100

-------------------------------------

DeepCNN | batch_size: 128 | epochs: 30
DeepCNN | batch_size: 128 | epochs: 50
DeepCNN | batch_size: 128 | epochs: 100
DeepCNN | batch_size: 256 | epochs: 30
DeepCNN | batch_size: 256 | epochs: 50
DeepCNN | batch_size: 256 | epochs: 100

-------------------------------------

DeepCNNCustom | batch_size: 128 | epochs: 30
DeepCNNCustom | batch_size: 128 | epochs: 50
DeepCNNCustom | batch_size: 128 | epochs: 100
DeepCNNCustom | batch_size: 256 | epochs: 30
DeepCNNCustom | batch_size: 256 | epochs: 50
DeepCNNCustom | batch_size: 256 | epochs: 100

-------------------------------------

VGGA11 | batch_size: 128 | epochs: 30
VGGA11 | batch_size: 128 | epochs: 50
VG

## Resultados

In [78]:
histories = glob.glob('{}/hist_*.json'.format(dirname))
histories

['/content/drive/My Drive/Ensino/UFRN 2016.1/2020.6/Aprendizado Profundo/projeto_final/models/hist_ShallowCNN_128_30.json',
 '/content/drive/My Drive/Ensino/UFRN 2016.1/2020.6/Aprendizado Profundo/projeto_final/models/hist_ShallowCNN_128_50.json',
 '/content/drive/My Drive/Ensino/UFRN 2016.1/2020.6/Aprendizado Profundo/projeto_final/models/hist_ShallowCNN_128_100.json',
 '/content/drive/My Drive/Ensino/UFRN 2016.1/2020.6/Aprendizado Profundo/projeto_final/models/hist_ShallowCNN_256_30.json',
 '/content/drive/My Drive/Ensino/UFRN 2016.1/2020.6/Aprendizado Profundo/projeto_final/models/hist_ShallowCNN_256_50.json',
 '/content/drive/My Drive/Ensino/UFRN 2016.1/2020.6/Aprendizado Profundo/projeto_final/models/hist_ShallowCNN_256_100.json',
 '/content/drive/My Drive/Ensino/UFRN 2016.1/2020.6/Aprendizado Profundo/projeto_final/models/hist_DeepCNN_128_30.json',
 '/content/drive/My Drive/Ensino/UFRN 2016.1/2020.6/Aprendizado Profundo/projeto_final/models/hist_DeepCNN_128_50.json',
 '/content/d

In [79]:
result = []
for f in histories:
    with open(f, "rb") as infile:
        result.append(json.load(infile))

In [80]:
with open("{}/merged_file.json".format(dirname), "w") as outfile:
     json.dump(result, outfile)

In [81]:
result[0]

{'id': 'ShallowCNN_128_30',
 'metrics': {'accuracy': 0.00719535443931818,
  'f1_metrics': 0.9912812113761902,
  'loss': 0.038720037788152695,
  'precision': 0.9919453263282776,
  'recall': 0.9909088015556335,
  'val_accuracy': 0.0005572583177126944,
  'val_f1_metrics': 0.5132482051849365,
  'val_loss': 2.0932767391204834,
  'val_precision': 0.5480329394340515,
  'val_recall': 0.50069659948349},
 'name': 'ShallowCNN',
 'params': {'batch_size': 128, 'epochs': 30, 'steps': 225}}

In [82]:
result[0]['id']

'ShallowCNN_128_30'

In [83]:
columns = [
  'id',
  'name',
  'accuracy',
  'f1_metrics',
  'loss',
  'precision',
  'recall',
  'val_accuracy',
  'val_f1_metrics',
  'val_loss',
  'val_precision',
  'val_recall',
  'batch_size',
  'epochs',
  'steps',
]

csv_data = []

for hist in result:
  data = [
    hist['id'],
    hist['name'],
    hist['metrics']['accuracy'],
    hist['metrics']['f1_metrics'],
    hist['metrics']['loss'],
    hist['metrics']['precision'],
    hist['metrics']['recall'],
    hist['metrics']['val_accuracy'],
    hist['metrics']['val_f1_metrics'],
    hist['metrics']['val_loss'],
    hist['metrics']['val_precision'],
    hist['metrics']['val_recall'],
    hist['params']['batch_size'],
    hist['params']['epochs'],
    hist['params']['steps'],
  ]
  csv_data.append(data)

In [84]:
df = pd.DataFrame(csv_data, columns=columns)

In [85]:
df.head()

,id,name,accuracy,f1_metrics,loss,precision,recall,val_accuracy,val_f1_metrics,val_loss,val_precision,val_recall,batch_size,epochs,steps
0,ShallowCNN_128_30,ShallowCNN,0.007195,0.991281,0.038720,0.991945,0.990909,0.000557,0.513248,2.093277,0.548033,0.500697,128,30,225
1,ShallowCNN_128_50,ShallowCNN,0.017103,0.994669,0.021875,0.994774,0.994531,0.001831,0.536337,2.116821,0.574442,0.523544,128,50,225
2,ShallowCNN_128_100,ShallowCNN,0.039425,0.996770,0.009355,0.996864,0.996656,0.003662,0.549105,2.199910,0.573171,0.523823,128,100,225
3,ShallowCNN_256_30,ShallowCNN,0.048910,0.997338,0.006182,0.997422,0.997213,0.004299,0.546227,2.272174,0.578078,0.536361,256,30,113
4,ShallowCNN_256_50,ShallowCNN,0.057020,0.996906,0.007185,0.996934,0.996830,0.005254,0.514100,2.346944,0.560864,0.513514,256,50,113


In [86]:
df['precision_max'] = df.groupby(['name'])['precision'].transform(max)

In [87]:
df_filter = df[df['precision'] != 0]
df_filter = df_filter[df_filter['precision_max'] == df_filter['precision']]
df_filter

,id,name,accuracy,f1_metrics,loss,precision,recall,val_accuracy,val_f1_metrics,val_loss,val_precision,val_recall,batch_size,epochs,steps,precision_max
5,ShallowCNN_256_100,ShallowCNN,0.071167,0.997839,0.004761,0.997944,0.997701,0.002189,0.525606,2.057211,0.569493,0.503483,256,100,113,0.997944
11,DeepCNN_256_100,DeepCNN,0.043610,0.994259,0.014876,0.994459,0.993974,0.011384,0.616898,2.570012,0.621567,0.605461,256,100,113,0.994459
23,VGGALRN11_256_100,VGGALRN11,0.119918,0.997717,0.005501,0.997979,0.997422,0.070891,0.561648,4.273236,0.589599,0.581220,256,100,113,0.997979
53,DeepCNNCustom_256_100,DeepCNNCustom,0.040132,0.985722,0.040009,0.986818,0.985649,0.008598,0.613123,2.202486,0.622566,0.605740,256,100,113,0.986818
59,VGGE19Custom_256_100,VGGE19Custom,0.190274,0.997700,0.004351,0.997840,0.997527,0.104048,0.602323,4.385242,0.621461,0.611591,256,100,113,0.997840
65,VGGD16Custom_256_100,VGGD16Custom,0.033011,0.964722,0.110778,0.967454,0.961893,0.012737,0.589995,2.493613,0.605798,0.564781,256,100,113,0.967454


In [88]:
grouped_df = df.groupby(['name'])
maximums = grouped_df.max()
maximums = maximums.reset_index()
maximums

,name,id,accuracy,f1_metrics,loss,precision,recall,val_accuracy,val_f1_metrics,val_loss,val_precision,val_recall,batch_size,epochs,steps,precision_max
0,DeepCNN,DeepCNN_256_50,0.043610,0.994259,0.149134,0.994459,0.993974,0.011384,0.630717,2.570012,0.626176,0.611870,256,100,225,0.994459
1,DeepCNNCustom,DeepCNNCustom_256_50,0.040132,0.985722,0.396511,0.986818,0.985649,0.008598,0.613123,2.202486,0.653329,0.605740,256,100,225,0.986818
2,ShallowCNN,ShallowCNN_256_50,0.071167,0.997839,0.038720,0.997944,0.997701,0.005254,0.549105,2.346944,0.578078,0.536361,256,100,225,0.997944
3,VGGA11,VGGA11_256_50,0.000000,0.000000,1.810604,0.000000,0.000000,0.000000,0.000000,1.811646,0.000000,0.000000,256,100,225,0.000000
4,VGGALRN11,VGGALRN11_256_50,0.119918,0.997717,0.117571,0.997979,0.997422,0.070891,0.583618,4.273236,0.610895,0.591530,256,100,225,0.997979
5,VGGB13,VGGB13_256_50,0.000000,0.000000,1.810553,0.000000,0.000000,0.000000,0.000000,1.812170,0.000000,0.000000,256,100,225,0.000000
6,VGGC16,VGGC16_256_50,0.000000,0.000000,1.810499,0.000000,0.000000,0.000000,0.000000,1.811707,0.000000,0.000000,256,100,225,0.000000
7,VGGD16,VGGD16_256_50,0.000000,0.000000,1.810617,0.000000,0.000000,0.000000,0.000000,1.812327,0.000000,0.000000,256,100,225,0.000000
8,VGGD16Custom,VGGD16Custom_256_50,0.033011,0.964722,0.722864,0.967454,0.961893,0.012737,0.589995,2.493613,0.689342,0.573697,256,100,225,0.967454
9,VGGE19,VGGE19_256_50,0.000000,0.000000,1.810546,0.000000,0.000000,0.000000,0.000000,1.812027,0.000000,0.000000,256,100,225,0.000000


In [89]:
df

,id,name,accuracy,f1_metrics,loss,precision,recall,val_accuracy,val_f1_metrics,val_loss,val_precision,val_recall,batch_size,epochs,steps,precision_max
0,ShallowCNN_128_30,ShallowCNN,0.007195,0.991281,0.038720,0.991945,0.990909,0.000557,0.513248,2.093277,0.548033,0.500697,128,30,225,0.997944
1,ShallowCNN_128_50,ShallowCNN,0.017103,0.994669,0.021875,0.994774,0.994531,0.001831,0.536337,2.116821,0.574442,0.523544,128,50,225,0.997944
2,ShallowCNN_128_100,ShallowCNN,0.039425,0.996770,0.009355,0.996864,0.996656,0.003662,0.549105,2.199910,0.573171,0.523823,128,100,225,0.997944
3,ShallowCNN_256_30,ShallowCNN,0.048910,0.997338,0.006182,0.997422,0.997213,0.004299,0.546227,2.272174,0.578078,0.536361,256,30,113,0.997944
4,ShallowCNN_256_50,ShallowCNN,0.057020,0.996906,0.007185,0.996934,0.996830,0.005254,0.514100,2.346944,0.560864,0.513514,256,50,113,0.997944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,VGGD16Custom_128_50,VGGD16Custom,0.005190,0.877137,0.343405,0.897089,0.858685,0.002508,0.565427,1.726418,0.620634,0.539705,128,50,225,0.967454
62,VGGD16Custom_128_100,VGGD16Custom,0.013634,0.928148,0.213213,0.936293,0.920443,0.006130,0.575574,2.084198,0.609905,0.555865,128,100,225,0.967454
63,VGGD16Custom_256_30,VGGD16Custom,0.022586,0.952184,0.144865,0.956832,0.948100,0.007682,0.560682,2.260318,0.603863,0.566174,256,30,113,0.967454
64,VGGD16Custom_256_50,VGGD16Custom,0.024835,0.960411,0.123774,0.963728,0.956947,0.008558,0.570547,2.347999,0.617576,0.573697,256,50,113,0.967454
